In [ ]:
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
import os
import random 
from dotenv import load_dotenv

# Configuración
load_dotenv(dotenv_path='../.env')

uri = os.getenv("NEO4J_URI")
# Fix para local vs docker
if "neo4j" in uri and "localhost" not in uri:
    print("⚠️ Detectado entorno local: Cambiando host 'neo4j' por 'localhost'")
    uri = uri.replace("neo4j", "localhost")

user = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")
AUTH = (user, password)

print(f"Conectando a: {uri}")
model = SentenceTransformer('all-MiniLM-L6-v2')

# --- DATASET PRODUCTOS ---
catalog_data = [
    # Laptops
    {"id": "L1", "nombre": "MacBook Air M2", "desc": "Laptop ligera Apple chip M2 13 pulgadas", "cat": "Laptops", "precio": 1200},
    {"id": "L2", "nombre": "Dell XPS 13", "desc": "Ultrabook Windows pantalla InfinityEdge", "cat": "Laptops", "precio": 1100},
    {"id": "L3", "nombre": "Asus ROG Zephyrus", "desc": "Laptop Gaming potente RTX 4060", "cat": "Laptops", "precio": 1600},
    {"id": "L4", "nombre": "Lenovo ThinkPad X1", "desc": "Empresarial ultrarresistente fibra de carbono", "cat": "Laptops", "precio": 1400},
    # Mouses
    {"id": "A1", "nombre": "Logitech MX Master 3S", "desc": "Mouse ergonómico productividad", "cat": "Accesorios", "precio": 100},
    {"id": "A2", "nombre": "Razer DeathAdder", "desc": "Mouse gaming alta precisión", "cat": "Accesorios", "precio": 60},
    # Otros
    {"id": "M1", "nombre": "Monitor LG Ultrawide", "desc": "Monitor 34 pulgadas curvo trabajo", "cat": "Monitores", "precio": 400},
    {"id": "H1", "nombre": "Sony WH-1000XM5", "desc": "Audífonos cancelación de ruido", "cat": "Audio", "precio": 350},
    {"id": "C1", "nombre": "Cargador Anker 100W", "desc": "Cargador rápido USB-C multipuerto", "cat": "Accesorios", "precio": 50},
]

relationships = [
    ("L1", "A1"), ("L1", "M1"), ("L1", "H1"), ("L1", "C1"),
    ("L2", "A1"), ("L2", "M1"), ("L2", "C1"),
    ("L3", "A2"), ("L3", "M1"), ("L3", "H1"),
]

# --- NUEVO: DATASET TIENDAS ---
tiendas = ["Tienda Central", "Sucursal Norte", "Venta Online"]

def setup_database():
    driver = GraphDatabase.driver(uri, auth=AUTH)
    with driver.session() as session:
        print("[ETL] 1. Limpiando Base de Datos...")
        session.run("MATCH (n) DETACH DELETE n")
        
        # Índices
        try: session.run("DROP INDEX productos_embeddings IF EXISTS")
        except: pass
        session.run("""
            CREATE VECTOR INDEX productos_embeddings IF NOT EXISTS
            FOR (p:Producto) ON (p.embedding)
            OPTIONS {indexConfig: {`vector.dimensions`: 384, `vector.similarity_function`: 'cosine'}}
        """)
        # Índice extra para búsquedas rápidas por ID
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (p:Producto) REQUIRE p.id IS UNIQUE")

        print("[ETL] 2. Insertando Productos...")
        for prod in catalog_data:
            text = f"{prod['nombre']} {prod['desc']} {prod['cat']}"
            vector = model.encode(text).tolist()
            
            # Insertar Producto
            query = """
            MERGE (c:Categoria {nombre: $cat})
            CREATE (p:Producto {id: $id, nombre: $nombre, descripcion: $desc, precio: $precio})
            CREATE (p)-[:PERTENECE_A]->(c)
            SET p.embedding = $vector
            """
            session.run(query, prod, vector=vector)

        print("[ETL] 3. Creando Tiendas y Asignando Stock...")
        for nombre_tienda in tiendas:
            # Crear la tienda
            session.run("MERGE (t:Tienda {nombre: $nombre})", nombre=nombre_tienda)
            
            # Asignar stock aleatorio a cada producto para esta tienda
            for prod in catalog_data:
                cantidad = random.randint(0, 10) # Entre 0 y 10 unidades
                if cantidad > 0: # Solo creamos relación si hay stock
                    session.run("""
                        MATCH (t:Tienda {nombre: $tienda})
                        MATCH (p:Producto {id: $pid})
                        MERGE (t)-[:TIENE_STOCK {cantidad: $cant}]->(p)
                    """, tienda=nombre_tienda, pid=prod['id'], cant=cantidad)

        print("[ETL] 4. Creando Relaciones de Compatibilidad...")
        for orig, dest in relationships:
            session.run("""
                MATCH (a:Producto {id: $pid1}), (b:Producto {id: $pid2})
                MERGE (a)-[:COMPATIBLE_CON]->(b)
            """, pid1=orig, pid2=dest)
            
    driver.close()
    print("--- ETL FINALIZADO CON ÉXITO ---")

if __name__ == "__main__":
    setup_database()

⚠️ Detectado entorno local: Cambiando host 'neo4j' por 'localhost'
Conectando a: bolt://localhost:7687


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 787.63it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


[ETL] 1. Limpiando Base de Datos...
[ETL] 2. Insertando Productos...
[ETL] 3. Creando Tiendas y Asignando Stock...
[ETL] 4. Creando Relaciones de Compatibilidad...
--- ETL FINALIZADO CON ÉXITO ---
